# First Notebook: Basic Walkthrough of XGBoost
@dzhang203
2019-05-21

See [official XGBoost python introduction](https://xgboost.readthedocs.io/en/latest/python/python_intro.html).

In [ ]:
import xgboost as xgb